# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 22 2023 2:33PM,257975,19,ACG-2102492849,ACG North America LLC,Released
1,Feb 22 2023 2:24PM,257974,18,EMERSA-9192358,"Emersa Waterbox, LLC",Released
2,Feb 22 2023 2:21PM,257973,19,MSP218811,"Methapharm, Inc.",Released
3,Feb 22 2023 1:42PM,257969,10,0086420659,ISDIN Corporation,Released
4,Feb 22 2023 1:42PM,257969,10,0086420655,ISDIN Corporation,Released
5,Feb 22 2023 1:42PM,257969,10,0086420634,ISDIN Corporation,Released
6,Feb 22 2023 1:42PM,257969,10,0086420633,ISDIN Corporation,Released
7,Feb 22 2023 1:42PM,257969,10,0086420636,ISDIN Corporation,Released
8,Feb 22 2023 1:42PM,257969,10,0086420653,ISDIN Corporation,Released
9,Feb 22 2023 1:42PM,257969,10,0086420656,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,257969,Released,64
33,257970,Released,2
34,257973,Released,1
35,257974,Released,1
36,257975,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257969,NaN,NaN,64.0
257970,NaN,NaN,2.0
257973,NaN,NaN,1.0
257974,NaN,NaN,1.0
257975,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257892,0.0,0.0,1.0
257907,0.0,0.0,2.0
257908,0.0,0.0,1.0
257910,0.0,0.0,2.0
257914,0.0,5.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257892,0,0,1
257907,0,0,2
257908,0,0,1
257910,0,0,2
257914,0,5,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257892,0,0,1
1,257907,0,0,2
2,257908,0,0,1
3,257910,0,0,2
4,257914,0,5,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257892,,,1
1,257907,,,2
2,257908,,,1
3,257910,,,2
4,257914,,5,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 22 2023 2:33PM,257975,19,ACG North America LLC
1,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC"
2,Feb 22 2023 2:21PM,257973,19,"Methapharm, Inc."
3,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation
67,Feb 22 2023 1:42PM,257968,10,ISDIN Corporation
102,Feb 22 2023 1:42PM,257967,10,ISDIN Corporation
145,Feb 22 2023 1:23PM,257970,19,"AdvaGen Pharma, Ltd"
147,Feb 22 2023 1:02PM,257966,10,"Nextsource Biotechnology, LLC"
148,Feb 22 2023 12:59PM,257965,10,"Citieffe, Inc."
151,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 22 2023 2:33PM,257975,19,ACG North America LLC,,,1
1,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC",,,1
2,Feb 22 2023 2:21PM,257973,19,"Methapharm, Inc.",,,1
3,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation,,,64
4,Feb 22 2023 1:42PM,257968,10,ISDIN Corporation,,,35
5,Feb 22 2023 1:42PM,257967,10,ISDIN Corporation,,,43
6,Feb 22 2023 1:23PM,257970,19,"AdvaGen Pharma, Ltd",,,2
7,Feb 22 2023 1:02PM,257966,10,"Nextsource Biotechnology, LLC",,,1
8,Feb 22 2023 12:59PM,257965,10,"Citieffe, Inc.",,2,1
9,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix,,2,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 2:33PM,257975,19,ACG North America LLC,1,,
1,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC",1,,
2,Feb 22 2023 2:21PM,257973,19,"Methapharm, Inc.",1,,
3,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation,64,,
4,Feb 22 2023 1:42PM,257968,10,ISDIN Corporation,35,,
5,Feb 22 2023 1:42PM,257967,10,ISDIN Corporation,43,,
6,Feb 22 2023 1:23PM,257970,19,"AdvaGen Pharma, Ltd",2,,
7,Feb 22 2023 1:02PM,257966,10,"Nextsource Biotechnology, LLC",1,,
8,Feb 22 2023 12:59PM,257965,10,"Citieffe, Inc.",1,2,
9,Feb 22 2023 12:55PM,257964,10,Yusen – 3D Matrix,,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 2:33PM,257975,19,ACG North America LLC,1,,
1,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC",1,,
2,Feb 22 2023 2:21PM,257973,19,"Methapharm, Inc.",1,,
3,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation,64,,
4,Feb 22 2023 1:42PM,257968,10,ISDIN Corporation,35,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 2:33PM,257975,19,ACG North America LLC,1.0,NaN,NaN
1,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Feb 22 2023 2:21PM,257973,19,"Methapharm, Inc.",1.0,NaN,NaN
3,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation,64.0,NaN,NaN
4,Feb 22 2023 1:42PM,257968,10,ISDIN Corporation,35.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 2:33PM,257975,19,ACG North America LLC,1.0,0.0,0.0
1,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Feb 22 2023 2:21PM,257973,19,"Methapharm, Inc.",1.0,0.0,0.0
3,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation,64.0,0.0,0.0
4,Feb 22 2023 1:42PM,257968,10,ISDIN Corporation,35.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5416776,221.0,9.0,0.0
12,257954,0.0,1.0,0.0
16,515913,2.0,0.0,0.0
18,257974,1.0,0.0,0.0
19,1805700,7.0,3.0,1.0
21,515891,2.0,0.0,0.0
22,257916,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5416776,221.0,9.0,0.0
1,12,257954,0.0,1.0,0.0
2,16,515913,2.0,0.0,0.0
3,18,257974,1.0,0.0,0.0
4,19,1805700,7.0,3.0,1.0
5,21,515891,2.0,0.0,0.0
6,22,257916,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,221.0,9.0,0.0
1,12,0.0,1.0,0.0
2,16,2.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,7.0,3.0,1.0
5,21,2.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,221.0
1,12,Released,0.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,9.0,1.0,0.0,0.0,3.0,0.0,0.0
Released,221.0,0.0,2.0,1.0,7.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,21,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,9.0,1.0,0.0,0.0,3.0,0.0,0.0
2,Released,221.0,0.0,2.0,1.0,7.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,21,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,9.0,1.0,0.0,0.0,3.0,0.0,0.0
2,Released,221.0,0.0,2.0,1.0,7.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()